In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt


In [5]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.7 MB/s eta 0:00:00a 0:00:01


In [8]:
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        # normalize specific to inception model
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

# load CIFAR10 dataset
cifar10_train = CIFAR10(root='/mnt_mount/labproject_data', train=True, download=True, transform=transform)
cifar10_test = CIFAR10(root='/mnt_mount/labproject_data', train=False, download=True, transform=transform)


100.0%


Extracting /mnt_mount/labproject_data/cifar-10-python.tar.gz to /mnt_mount/labproject_data
Files already downloaded and verified


In [10]:
dataloader_1 = torch.utils.data.DataLoader(cifar10_train, batch_size=100, shuffle=False, num_workers=1)
dataloader_2 = torch.utils.data.DataLoader(cifar10_test, batch_size=100, shuffle=False, num_workers=1)

In [11]:
from torchvision.models import inception_v3
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np


# get embedding net
def get_embedding_net():
    model = inception_v3(pretrained=True)
    model.fc = torch.nn.Identity()  # replace the classifier with identity to get features
    model.eval()
    return model.to('cuda' if torch.cuda.is_available() else 'cpu')

# extract features
def extract_features(dataloader, model):
    features = []
    with torch.no_grad():
        for data, _ in dataloader:
            data = data.to('cuda' if torch.cuda.is_available() else 'cpu')
            features.append(model(data))
    return torch.cat(features).cpu().numpy()


embedding_net = get_embedding_net()

features1 = extract_features(dataloader_1, embedding_net)
features2 = extract_features(dataloader_2, embedding_net)


/mnt/miniconda3/envs/labproject/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/miniconda3/envs/labproject/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/mnt/miniconda3/envs/labproject/lib/python3.9/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [12]:
features1.shape, features2.shape

((50000, 2048), (10000, 2048))

In [16]:
from labproject.metrics.sliced_wasserstein import sliced_wasserstein_distance

swd = sliced_wasserstein_distance(torch.from_numpy(features1)[:10000], torch.from_numpy(features2), num_projections=1000)